In [1]:
import numpy as np
import pandas as pd
import main
las_data = pd.read_csv(main.csv_las_out_full_path, delimiter=';')
las_data.head()

,well_name,STRT,STOP,DEPT,aps,rp,kp,kgl,kpr,kvo,kng,lit,satur,UWI
0,224st4,2729.2,2929.4,2729.2,0.22,8.4,0.095,0.329,0.1,1.0,0.0,0.0,0.0,314_103ST2
1,224st4,2729.2,2929.4,2729.3,0.22,8.4,0.095,0.329,0.1,1.0,0.0,0.0,0.0,314_103ST2
2,224st4,2729.2,2929.4,2729.4,0.22,8.4,0.095,0.329,0.1,1.0,0.0,0.0,0.0,314_103ST2
3,224st4,2729.2,2929.4,2729.5,0.22,8.4,0.095,0.329,0.1,1.0,0.0,0.0,0.0,314_103ST2
4,224st4,2729.2,2929.4,2729.6,0.22,8.4,0.095,0.329,0.1,1.0,0.0,0.0,0.0,314_103ST2


In [2]:
del las_data['STRT']
del las_data['STOP']
del las_data['rp']
del las_data['kpr']
del las_data['kvo']
del las_data['kng']
del las_data['lit']
del las_data['satur']
del las_data['kp']
las_data.head()

,well_name,DEPT,aps,kgl,UWI
0,224st4,2729.2,0.22,0.329,314_103ST2
1,224st4,2729.3,0.22,0.329,314_103ST2
2,224st4,2729.4,0.22,0.329,314_103ST2
3,224st4,2729.5,0.22,0.329,314_103ST2
4,224st4,2729.6,0.22,0.329,314_103ST2


In [3]:
las_data.replace('314_', '', regex=True, inplace=True)
#del las_data['well_name']
las_data = las_data.rename(columns={'well_name': 'WELL_NAME'})
las_data = las_data.rename(columns={'UWI': 'WELL_NAME_UWI'})
las_data = las_data.rename(columns={'kgl': 'KGL'})
las_data = las_data.rename(columns={'DEPT': 'DEPTH'})
las_data.head(n=2)

,WELL_NAME,DEPTH,aps,KGL,WELL_NAME_UWI
0,224st4,2729.2,0.22,0.329,103ST2
1,224st4,2729.3,0.22,0.329,103ST2


In [4]:
real_top = pd.read_csv('real_data_test\\top_df.csv', delimiter=';')
real_top = real_top.rename(columns={'WELL_NAME': 'WELL_NAME_UWI'})
del real_top['X']
del real_top['Y']
real_top.head(n=2)

,WELL_NAME_UWI,DEPTH_TOP
0,100,2548.81
1,101,2556.23


In [5]:
real_bottom = pd.read_csv('real_data_test\\bottom_df.csv', delimiter=';')
real_bottom = real_bottom.rename(columns={'WELL_NAME': 'WELL_NAME_UWI'})
del real_bottom['X']
del real_bottom['Y']
real_bottom.head(n=2)

,WELL_NAME_UWI,DEPTH_BOTTOM
0,100,2564.81
1,101,2571.71


In [6]:
water_content_data = pd.read_csv(main.data_dir + "\\water_content.csv", delimiter=';')
water_content_data = water_content_data.rename(columns={'Скважина': main.keys_dict[main.kid_well]})
water_content_data = water_content_data.rename(columns={' Начальная обводненность ': 'WC'})
water_content_data = water_content_data.rename(columns={'Дата запуска': 'date_start'})
del water_content_data['date_start']
water_content_data = water_content_data.rename(columns={'well_name': 'WELL_NAME'})
water_content_data.head(n=2)

,WELL_NAME,WC
0,380,0.49
1,379,0.30


In [7]:
# данные посчитанные на карте
wc_calc_data = pd.read_csv('fake_data\\map_wc_test_calc_res.csv', delimiter=';')
del wc_calc_data['Y']
del wc_calc_data['X']
wc_calc_data['WELL_NAME'] = wc_calc_data['WELL_NAME'].astype('str')
wc_calc_data = wc_calc_data.drop(wc_calc_data[wc_calc_data['WC'] > 1].index)
wc_calc_data.head()

,WELL_NAME,WC
0,380,0.392981
1,379,0.286590
2,370,0.231697
3,417,0.195822
4,389,0.277036


In [8]:
wc_calc_data = wc_calc_data.rename(columns={'WC': 'WC_CALC'})

In [9]:
print(len(las_data['WELL_NAME'].value_counts()))
print(len(real_bottom['WELL_NAME_UWI'].value_counts()))
print(len(real_top['WELL_NAME_UWI'].value_counts()))
print(len(water_content_data['WELL_NAME'].value_counts()))

223
230
230
180


In [10]:
wc_calc_data =pd.merge(wc_calc_data, water_content_data, on='WELL_NAME')
wc_calc_data.head()

,WELL_NAME,WC_CALC,WC
0,380,0.392981,0.49
1,379,0.286590,0.30
2,370,0.231697,0.13
3,417,0.195822,0.08
4,389,0.277036,0.22


In [11]:
real_data = pd.merge(wc_calc_data, las_data, on='WELL_NAME')
real_data = pd.merge(real_data, real_top, on='WELL_NAME_UWI')
real_data = pd.merge(real_data, real_bottom, on='WELL_NAME_UWI') 
real_data.head()

,WELL_NAME,WC_CALC,WC,DEPTH,aps,KGL,WELL_NAME_UWI,DEPTH_TOP,DEPTH_BOTTOM
0,380,0.392981,0.49,2593.0,0.01,0.444,178,2560.32,2586.48
1,380,0.392981,0.49,2593.1,0.01,0.444,178,2560.32,2586.48
2,380,0.392981,0.49,2593.2,0.01,0.444,178,2560.32,2586.48
3,380,0.392981,0.49,2593.3,0.01,0.444,178,2560.32,2586.48
4,380,0.392981,0.49,2593.4,0.01,0.444,178,2560.32,2586.48


In [12]:
print(len(real_data['WELL_NAME'].value_counts()))
real_data.info()

164
<class 'pandas.core.frame.DataFrame'>
Int64Index: 341815 entries, 0 to 341814
Data columns (total 9 columns):
WELL_NAME        341815 non-null object
WC_CALC          341815 non-null float64
WC               341815 non-null float64
DEPTH            341815 non-null float64
aps              149205 non-null float64
KGL              180953 non-null float64
WELL_NAME_UWI    341815 non-null object
DEPTH_TOP        341815 non-null float64
DEPTH_BOTTOM     339886 non-null float64
dtypes: float64(7), object(2)
memory usage: 26.1+ MB


In [13]:
real_data_na = real_data.dropna()
print(len(real_data['WELL_NAME'].value_counts()))
real_data.info()

164
<class 'pandas.core.frame.DataFrame'>
Int64Index: 341815 entries, 0 to 341814
Data columns (total 9 columns):
WELL_NAME        341815 non-null object
WC_CALC          341815 non-null float64
WC               341815 non-null float64
DEPTH            341815 non-null float64
aps              149205 non-null float64
KGL              180953 non-null float64
WELL_NAME_UWI    341815 non-null object
DEPTH_TOP        341815 non-null float64
DEPTH_BOTTOM     339886 non-null float64
dtypes: float64(7), object(2)
memory usage: 26.1+ MB


In [14]:
import lasio
import re
def load_and_convert_to_interp(dev_path, well_name):
    f = open(dev_path + well_name + '.dev', 'r')
    well_num = 0
    md = []
    abs = []
    for line in f.readlines():
        if well_num > 16:
            # list = line.split(' ')
            # print(list)
            numbers = re.findall(r'[-]?[0-9]+.[0-9]+', line)
            md.append(float(numbers[0]))
            abs.append(float(numbers[3]))
        well_num = well_num +1
    f.close()
    f_spline = interpolate.interp1d(abs, md, kind = 'slinear', bounds_error=False)
    return f_spline

Загружаем переход от md в abs

In [15]:
from scipy import interpolate
dev_path = "..\\tasks\\task 6\\data\\trace\\"
well_name_list = real_data_na['WELL_NAME_UWI'].value_counts().index.tolist()
well_name = well_name_list[0]
f_spline = load_and_convert_to_interp(dev_path, well_name)
print(f_spline(0))

107.8


In [23]:
from scipy import interpolate
well_name_list = real_data_na['WELL_NAME_UWI'].value_counts().index.tolist()
x_values = []
y_values = []
y_values_calc = []
well_values = []
count_val = 100
well_count = 0
for well_name in well_name_list:
    if well_count%20 == 0:
        print(well_count, ' of ', len(well_name_list))
    data_well =  real_data_na[real_data_na['WELL_NAME_UWI'] == well_name]
    f_spline = load_and_convert_to_interp(dev_path, well_name)
    bottom = f_spline(data_well['DEPTH_BOTTOM'].tolist()[0])
    top = f_spline(data_well['DEPTH_TOP'].tolist()[0])
    data_well_by_bound = data_well[(data_well['DEPTH'] >= top) & (data_well['DEPTH'] <= bottom)]
    x_arr = data_well_by_bound['DEPTH']
    y_arr = data_well_by_bound['KGL']
    
    # print ('length of array depth', len(x_arr))
    # print(len(x_arr), len(y_arr))
    if len(x_arr) < 10:
        continue
    f_spline = interpolate.interp1d(x_arr, y_arr, kind = 'slinear')
    h_start = data_well_by_bound['DEPTH'].min()
    h_end = data_well_by_bound['DEPTH'].max()
    # print(h_start, h_end, top, bottom)
    h_step = (h_end - h_start)/count_val
    x_temp = []
    i = 0
    while(i < count_val):
        x_temp.append(float(f_spline(h_start + i * h_step)))
        i = i + 1
    x_values.append(x_temp)  
    y_values.append(data_well['WC'].tolist()[0])
    y_values_calc.append(data_well['WC_CALC'].tolist()[0])
    well_values.append(data_well['WELL_NAME_UWI'].tolist()[0])
    well_count = well_count + 1
print('end fill')

0  of  132
20  of  132
40  of  132
60  of  132
80  of  132
100  of  132
120  of  132
end fill


In [24]:
print(len(y_values))
print(len(y_values_calc))

129
129


In [25]:
save_x_kgl = pd.DataFrame(x_values, columns=['kgl' + str(z) for z in range(0, len(x_values[0]))])
save_y_wc_real_kgl = pd.DataFrame(y_values, columns=['wc'])
save_y_wc_calc_kgl = pd.DataFrame(y_values_calc, columns=['wc'])
well_kgl = pd.DataFrame(well_values, columns=['well_name'])

save_xy_real_kgl = pd.merge(well_kgl, save_y_wc_real_kgl, left_index = True, right_index = True)
save_xy_real_kgl = pd.merge(save_xy_real_kgl, save_x_kgl, left_index = True, right_index = True)
# save_xy_real.to_csv('fake_data/_kgl_real_wc.csv', sep = ';', index = False)

save_xy_calc_kgl = pd.merge(well_kgl, save_y_wc_calc_kgl, left_index = True, right_index = True)
save_xy_calc_kgl = pd.merge(save_xy_calc_kgl, save_x_kgl, left_index = True, right_index = True)
# save_xy_calc.to_csv('fake_data/_kgl_calc_wc.csv', sep = ';', index = False)

In [26]:
from scipy import interpolate
well_name_list = real_data_na['WELL_NAME_UWI'].value_counts().index.tolist()
x_values = []
y_values = []
y_values_calc = []
well_values = []
count_val = 100
well_count = 0
for well_name in well_name_list:
    if well_count%20 == 0:
        print(well_count, ' of ', len(well_name_list))
    data_well =  real_data_na[real_data_na['WELL_NAME_UWI'] == well_name]
    f_spline = load_and_convert_to_interp(dev_path, well_name)
    bottom = f_spline(data_well['DEPTH_BOTTOM'].tolist()[0])
    top = f_spline(data_well['DEPTH_TOP'].tolist()[0])
    data_well_by_bound = data_well[(data_well['DEPTH'] >= top) & (data_well['DEPTH'] <= bottom)]
    x_arr = data_well_by_bound['DEPTH']
    y_arr = data_well_by_bound['aps']
    
    # print ('length of array depth', len(x_arr))
    # print(len(x_arr), len(y_arr))
    if len(x_arr) < 10:
        continue
    f_spline = interpolate.interp1d(x_arr, y_arr, kind = 'slinear')
    h_start = data_well_by_bound['DEPTH'].min()
    h_end = data_well_by_bound['DEPTH'].max()
    # print(h_start, h_end, top, bottom)
    h_step = (h_end - h_start)/count_val
    x_temp = []
    i = 0
    while(i < count_val):
        x_temp.append(float(f_spline(h_start + i * h_step)))
        i = i + 1
    x_values.append(x_temp)  
    y_values.append(data_well['WC'].tolist()[0])
    y_values_calc.append(data_well['WC_CALC'].tolist()[0])
    well_values.append(data_well['WELL_NAME_UWI'].tolist()[0])
    well_count = well_count + 1
print('end fill')

0  of  132
20  of  132
40  of  132
60  of  132
80  of  132
100  of  132
120  of  132
end fill


In [27]:
print(len(y_values))
print(len(y_values_calc))

129
129


In [28]:
save_x_aps = pd.DataFrame(x_values, columns=['aps' + str(z) for z in range(0, len(x_values[0]))])
save_y_wc_real_aps = pd.DataFrame(y_values, columns=['wc'])
save_y_wc_calc_aps = pd.DataFrame(y_values_calc, columns=['wc'])
well_aps = pd.DataFrame(well_values, columns=['well_name'])

save_xy_real_aps = pd.merge(well_aps, save_y_wc_real_aps, left_index = True, right_index = True)
save_xy_real_aps = pd.merge(save_xy_real_aps, save_x_aps, left_index = True, right_index = True)
# save_xy_real.to_csv('fake_data/_kgl_real_wc.csv', sep = ';', index = False)

save_xy_calc_aps = pd.merge(well_aps, save_y_wc_calc_aps, left_index = True, right_index = True)
save_xy_calc_aps = pd.merge(save_xy_calc_aps, save_x_aps, left_index = True, right_index = True)
# save_xy_calc.to_csv('fake_data/_kgl_calc_wc.csv', sep = ';', index = False)

In [31]:
save_xy_real_aps_kgl = pd.merge(save_xy_real_aps, save_xy_real_kgl, on=['well_name'])
save_xy_real_aps_kgl.to_csv('fake_data/_aps_kgl_real_wc.csv', sep = ';', index = False)

In [32]:
save_xy_calc_aps_kgl = pd.merge(save_xy_calc_aps, save_xy_calc_kgl, on=['well_name'])
save_xy_calc_aps_kgl.to_csv('fake_data/_aps_kgl_calc_wc.csv', sep = ';', index = False)